# Introduction

*Welcome to MLServe.com!*

---
In this notebook you will learn how to:

1. Setup your notebook so that you can import MLServeClient and register/login to your account
2. Train and deploy a model
3. Make predictions using the MLServe.com client or a direct request
4. Capturing metrics like requests, latency and requests per second

# Preparation

### Installing the client

From this notebook (or in your requirements.txt of the environment) run

```sh
pip install mlserve-sdk
```

### Setting up credentials
If you use the standard authentication method (username, password) then create a .env file and paste your credentials

```txt
USERNAME=your-username
TOKEN=your-password
```

If you intend to register through google auth then you don't need to do anything!

In [1]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from mlserve_sdk.client import MLServeClient
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
# -----------------------------
# 1a. Connect to MLServe - Standard auth
# -----------------------------
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("TOKEN")

client = MLServeClient()
client.login(USERNAME, PASSWORD)

In [4]:
# -----------------------------
# 1b. Connect to MLServe - Google auth
# -----------------------------
client = MLServeClient()
client.google_login()

In [6]:
# -----------------------------
# 2. Train a sample model
# -----------------------------
data = load_iris(as_frame=True)
X = data['data']
y = data['target']
model = LogisticRegression(max_iter=200)
model.fit(X, y)
score=model.score(X, y)

In [7]:
# -----------------------------
# 3. Deploy model
# -----------------------------
feats=data['feature_names']
client.deploy(
    model=model,
    name="iris",
    version="v1",
    features=feats,
    background_df=X,
    task_type='classification',
    metrics={'accuracy':score}
)

{'predict_url': 'https://mlserve.com/api/v1/predict/iris/v1'}

In [27]:
# -----------------------------
# 4a. Predict (specific version - SDK)
# -----------------------------
iris_data = {
    "inputs": [
        [5.1, 3.5, 1.4, 0.2],  # Iris-setosa
    ]
}
preds = client.predict("iris", "v1", iris_data, explain=True)
print("Predictions:", preds)

Predictions: {'version': 'v1', 'predictions': [0], 'explanations': [[{'feature': 'petal length (cm)', 'value': 1.4, 'shap_value': 5.623, 'impact': 'positive'}, {'feature': 'petal width (cm)', 'value': 0.2, 'shap_value': 1.021, 'impact': 'positive'}, {'feature': 'sepal width (cm)', 'value': 3.5, 'shap_value': 0.412, 'impact': 'positive'}]], 'prediction_ids': ['d0d9063f-ee25-4e0e-ad88-95e923a4baf3']}


In [32]:
# -----------------------------
# 4b. Predict (directly on the endpoint)
# -----------------------------

import requests
import json

# ----------------------------
# Configuration
# ----------------------------
SERVER_URL = "https://mlserve.com"
MODEL_NAME = "iris"
MODEL_VERSION = "v1"
TOKEN = "YOUR_ACCESS_TOKEN_HERE"  # obtained from login or google_login
# Example input data — structure depends on your model
payload = {
    "inputs": [
        [5.1, 3.5, 1.4, 0.2],  # Iris-setosa
    ]
}

# ----------------------------
# Direct API call
# ----------------------------
url = f"{SERVER_URL}/api/v1/predict/{MODEL_NAME}/{MODEL_VERSION}"
headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, data=json.dumps(payload))

# ----------------------------
# Response handling
# ----------------------------
if response.status_code == 200:
    result = response.json()
    print("✅ Predictions:")
    print(json.dumps(result, indent=2))
else:
    print(f"❌ Error {response.status_code}: {response.text}")

✅ Predictions:
{
  "version": "v1",
  "prediction_ids": [
    "2fa19810-7734-4d0d-91cb-9244e1590a21"
  ],
  "predictions": [
    0
  ],
  "explanations": [
    []
  ]
}


In [33]:
# -----------------------------
# 5. Fetch Basic Metrics
# -----------------------------
metrics = client.get_metrics("iris", "v1", as_dataframe=True)
metrics

,requests,predictions,throughput_rps,prediction_rps,avg_latency_ms,p50_latency_ms,p95_latency_ms,p99_latency_ms,avg_latency_per_element_ms,p50_latency_per_element_ms,p95_latency_per_element_ms,p99_latency_per_element_ms,error_rate
timestamp,,,,,,,,,,,,,
2025-10-05 07:00:00+00:00,20,20,0.005556,0.005556,32.976645,33.843135,39.636271,65.179446,32.976645,33.843135,39.636271,65.179446,0.0
2025-10-05 08:00:00+00:00,1,1,0.000278,0.000278,76.300269,76.300269,76.300269,76.300269,76.300269,76.300269,76.300269,76.300269,0.0
